In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')
plt.style.use(['seaborn-bright','dark_background'])

In [9]:
data=pd.read_csv('churn_prediction_simple.csv')

In [10]:
data=data.dropna()

In [11]:
#seperating independent and dependent variable
X=data.drop(columns=['churn','customer_id'])
Y=data['churn']

In [12]:
#scaling
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X)

In [6]:
#splitting
from sklearn.model_selection import train_test_split as tts
x_train,x_test,y_train,y_test=tts(scaled_X,Y,train_size=0.80,stratify=None)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((17653, 19), (4414, 19), (17653,), (4414,))

In [14]:
from sklearn.tree import DecisionTreeClassifier as DTC
classifier=DTC(class_weight='balanced')
classifier=DTC()

In [15]:
classifier.fit(x_train,y_train)
predicted_values=classifier.predict(x_train)

In [16]:
#finding out perfomance
predicted_values[:30]

array([0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0], dtype=int64)

In [17]:
#evaluation metrics
from sklearn.metrics import classification_report
print(classification_report(y_train,predicted_values))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     14264
           1       1.00      1.00      1.00      3389

    accuracy                           1.00     17653
   macro avg       1.00      1.00      1.00     17653
weighted avg       1.00      1.00      1.00     17653



In [18]:
predicted_values=classifier.predict(x_test)
print((classification_report(y_test,predicted_values)))

              precision    recall  f1-score   support

           0       0.86      0.85      0.86      3529
           1       0.43      0.45      0.44       885

    accuracy                           0.77      4414
   macro avg       0.64      0.65      0.65      4414
weighted avg       0.77      0.77      0.77      4414



In [19]:
!pip install graphviz

In [21]:
from sklearn.tree import export_graphviz
export_graphviz(decision_tree=classifier,out_file='tree_viz',
               max_depth=None,feature_names=X.columns,
               label=None,impurity=False)

In [25]:
from graphviz import render
render(filepath ='tree_viz',format='png',engine='neato')

ExecutableNotFound: failed to execute ['dot', '-Kneato', '-Tpng', '-O', 'tree_viz'], make sure the Graphviz executables are on your systems' PATH

In [27]:
##avoid over fitting usuing physical charecteristics using pruninh
classifier= DTC()
classifier.fit(x_train,y_train)

DecisionTreeClassifier()

In [40]:
#max depth
from sklearn.metrics import f1_score
def calc_score(model,x1,y1,x2,y2):
    model.fit(x1,y1)
    predict=model.predict(x1)
    f1=f1_score(y1,predict)
    predict=model.predict(x2)
    f2=f1_score(y2,predict)
    
    return f1,f2    
    

In [41]:
def effect(train_score,test_score,x_axis,title):
    plt.figure(figsize=(5,5),dpi=120)
    plt.plot(x_axis,train_score,color='red',label='train_Score')
    plt.plot(x_axis,train_score,color='red',label='test_Score')
    plt.title(title)
    plt.legend()
    plt.xlabel("Parameter value")
    
    plt.ylabe("f1 score")
    plt.show()
    

In [42]:
max_depth=[i for i in range (1,50)]
train=[]
test=[]

for i in max_depth:
    model=DTC(class_weight='balanced',max_depth=i,random_state=42)
    f1,f2=calc_score(model,x_train,y_train,x_test,y_test)
    train.append(f1)
    test.append(f2)
    